In [1]:
import torch
print(torch.cuda.is_available() , torch.__version__)

False 2.2.0+cpu


In [1]:
from src import API
API.Trainer.main(stage = 0 , resume = 0 , checkname= 1)

24-05-29 17:58:28|MOD:display     |: Model Specifics:
24-05-29 17:58:28|MOD:display     |: Start Process [Data] at Wed May 29 17:58:28 2024!


--Process Queue : Data + Fit + Test
--Model_name is set to tra_day_ShortTest!
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=10) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=10,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=10) , display batch progress bar
Callback : StatusDisplay(verbosity=10) , display epoch and event information
Callback : SpecCB_TRA() , in TRA fill [y] [hist_loss] in batch_data.kwargs , update hist_loss in data.buffer
{'random_seed': None,
 'model_name': 'tr

24-05-29 17:58:31|MOD:display     |: Finish Process [Data], Cost 2.4 Secs
24-05-29 17:58:31|MOD:display     |: Start Process [Fit] at Wed May 29 17:58:31 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


Train Ep#  0 loss : 1.02465:   2%|▏         | 1/48 [00:00<00:12,  3.80it/s]

score function of [spearman] calculated and success!
loss function of [mse] calculated and success!
penalty function of [hidden_corr] calculated and success!


Valid Ep#  0 score : 0.01377: 100%|██████████| 12/12 [00:00<00:00, 12.69it/s] 
FirstBite Ep#  0 : loss  1.02193, train 0.00261, valid-0.01300, best-0.0130, lr1.3e-03
Valid Ep#  1 score : -0.00135: 100%|██████████| 12/12 [00:00<00:00, 13.57it/s]
FirstBite Ep#  1 : loss  1.01572, train 0.01530, valid 0.00873, best 0.0087, lr2.5e-03
Valid Ep#  2 score : 0.00882: 100%|██████████| 12/12 [00:00<00:00, 14.36it/s] 
FirstBite Ep#  2 : loss  1.01352, train 0.03089, valid 0.01611, best 0.0161, lr3.8e-03
Valid Ep#  3 score : -0.01170: 100%|██████████| 12/12 [00:00<00:00, 15.84it/s]
FirstBite Ep#  3 : loss  1.01302, train 0.03806, valid 0.03524, best 0.0352, lr5.0e-03
24-05-29 17:59:14|MOD:display     |: tra_day_ShortTest #0 @20170103|FirstBite Ep#  4 Max Epoch|Train 0.0381 Valid 0.0352 BestVal 0.0352|Cost  0.7Min,  8.2Sec/Ep
24-05-29 17:59:14|MOD:display     |: Finish Process [Fit], Cost 0.0 Hours, 0.7 Min/model, 10.8 Sec/Epoch
24-05-29 17:59:14|MOD:display     |: Start Process [Test] at Wed May 2

                    hook_name  num_calls  total_time  avg_time
10             on_train_batch        192   34.800170  0.181251
27              on_test_batch        282   15.573823  0.055226
15        on_validation_batch         48    2.700895  0.056269
4          on_fit_model_start          1    2.096335  2.096335
24        on_test_model_start          1    1.148355  1.148355
25   on_test_model_type_start          3    0.023086  0.007695
21           on_fit_model_end          1    0.022537  0.022537
17    on_validation_epoch_end          4    0.019749  0.004937
8          on_before_backward        192    0.015623  0.000081
12         on_train_epoch_end          4    0.002507  0.000627
26        on_test_batch_start        282    0.002506  0.000009
7        on_train_batch_start        192    0.002009  0.000010
20       on_before_save_model          1    0.000520  0.000520
23              on_test_start          1    0.000013  0.000013
28          on_test_batch_end        282    0.000000  0

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
 
class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_model, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.sqrt_d_model = math.sqrt(d_model)
 
    def forward(self, Q, K, V, mask=None):
        # 计算点积
        scores = torch.matmul(Q, K.transpose(-2, -1)) / self.sqrt_d_model
        
        # 掩码处理，如掩蔽未来信息在自注意力中的应用
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        # 应用softmax函数
        attention_weights = F.softmax(scores, dim=-1)
        
        # 使用dropout增加模型的泛化能力
        attention_weights = self.dropout(attention_weights)
        
        # 加权求和得到输出
        output = torch.matmul(attention_weights, V)
        

In [59]:
import torch
bs = 20
hidden_dim = 16
h2 = 8
factor_num = 10

a = torch.rand(bs , hidden_dim)
q = torch.nn.Parameter(torch.rand(h2))
k = torch.nn.Linear(hidden_dim,h2,bias=False)(a)
v = torch.nn.Linear(hidden_dim,h2,bias=False)(a)

In [62]:
a = torch.nn.ReLU()((q * k) / q.norm() / k.norm(dim=-1,keepdim=True)) + 1e-6
a = a / a.sum(-1 , keepdim=True)
(a * v).sum(0).reshape(1,-1)

In [1]:
%run "src/nn/factorVAE.py"

Hidden Features shape : torch.Size([40, 32])
Portoflio shape : torch.Size([40, 100])
mu_post shape : torch.Size([1, 64]) , sigma_post shape : torch.Size([1, 64])
A post factor sample : tensor([[ 1.0287,  0.6462, -0.0442,  1.1449, -0.3113,  1.5007, -0.6798,  2.5814,
          1.5795,  1.4307,  0.6464,  0.9147,  0.3010,  0.8725,  1.3673,  1.0741,
          1.0826,  0.0390,  1.6443,  1.0617,  1.1597,  1.4367,  1.7134, -0.5356,
         -0.6769, -1.5016,  1.0472,  1.2781,  0.1810,  1.5498,  2.5867,  1.1622,
          0.4741,  0.7150,  1.0427, -0.2219,  1.0836, -0.8425,  0.5996,  1.2462,
          0.7252,  1.2713,  0.8382,  1.7843,  1.5991,  0.0106,  1.2709, -1.1145,
          0.9098,  1.1036,  2.3179,  1.6863,  2.7604,  0.8293,  1.1856,  1.1063,
         -0.9258,  0.4866,  3.0375,  0.4554, -0.1498,  1.6770,  1.8241,  1.0082]],
       grad_fn=<AddBackward0>) 
Beta shape : torch.Size([40, 64])
mu_alpha shape : torch.Size([40, 1]) , sigma_alpha shape : torch.Size([40, 1])
stock_returns shape 

In [51]:
q.norm(dim = 1).shape

torch.Size([20])

In [2]:
from src.environ import PATH
PATH.read_yaml('./configs/model_lgbm.yaml')

{'seqlens': [{'day': 30, '30m': 30, 'dms': 30}],
 'objective': ['regression'],
 'linear_tree': [True],
 'learning_rate': [0.3],
 'lambda_l2': ['1e-05'],
 'alpha': ['1e-07'],
 'num_leaves': [31],
 'max_depth': [6],
 'min_sum_hessian_in_leaf': [1],
 'feature_fraction': [0.6],
 'bagging_fraction': [0.75],
 'force_col_wise': [True],
 'monotone_constraints': [1]}

In [7]:
from src import API
API.DataAPI.reconstruct_train_data()

Use device name: NVIDIA GeForce RTX 4090
predict is False , Data Processing start!
4 datas :['y', 'day', '30m', 'risk']
y blocks loading start!
 --> labels blocks reading [ret10_lag] DataBase...... finished! Cost 18.86 secs
 --> labels blocks reading [ret20_lag] DataBase...... finished! Cost 20.08 secs
 --> labels blocks merging (2)...... finished! Cost 2.88 secs
 --> models blocks reading [risk_exp] DataBase...... finished! Cost 67.78 secs
y blocks loading finished! Cost 120.52 secs
y blocks process...... finished! Cost 45.61 secs
y blocks masking...... finished! Cost 0.81 secs
y blocks saving ...... finished! Cost 3.76 secs
y blocks norming...... finished! Cost 0.00 secs
y finished! Cost 170.98 Seconds
--------------------------------------------------------------------------------
day blocks loading start!
 --> trade blocks reading [day] DataBase...... finished! Cost 28.95 secs
day blocks loading finished! Cost 28.98 secs
day blocks process...... finished! Cost 3.27 secs
day blocks 